In [11]:
# importamos las librerías necesarias
import sim
import numpy as np
import sys
import time
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [12]:
def connect(port):
# Establece la conexión a VREP
# port debe coincidir con el puerto de conexión en VREP
# retorna el número de cliente o -1 si no puede establecer conexión
    sim.simxFinish(-1) # just in case, close all opened connections
    clientID=sim.simxStart('127.0.0.1',port,True,True,2000,5) # Conectarse
    if clientID == 0: print("conectado a", port)
    else: print("no se pudo conectar")
    return clientID


In [13]:
# Conectarse al servidor de VREP
# *** ejecutar cada vez que se reinicia la simulación ***
clientID = connect(19999)

conectado a 19999


In [14]:
#Guardamos el handle de la cámara del robot (el 'Vision_sensor')
error_cam, camara = sim.simxGetObjectHandle(clientID, 'Camara', sim.simx_opmode_oneshot_wait)
  
#Capturamos un frame para activar la cámara y esperamos 1 segundo:
_, resolution, image = sim.simxGetVisionSensorImage(clientID, camara, 0, sim.simx_opmode_streaming)
time.sleep(1)
  
while(1):
    #Capturamos un frame de la cámara del robot. Guardamos la imagen y su resolución:
    _, resolution, image = sim.simxGetVisionSensorImage(clientID, camara, 0, sim.simx_opmode_buffer)
     
    #Modificaremos esta imagen para que OpenCV pueda tratarla:
    img = np.array(image, dtype = np.uint8) #La convertimos a un array de numpy
    img.resize([resolution[0], resolution[1], 3]) #Cambiamos sus dimensiones
    img = np.rot90(img,2) #La rotamos 90 grados para enderezarla
    img = np.fliplr(img) #La invertimos
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR) #Cambiamos su espacio de color de RGB a BGR
     
    #Mostramos la imagen:
    cv2.imshow('Image', img)
     
    #Se sale con ESC.
    tecla = cv2.waitKey(5) & 0xFF
    if tecla == 27:
        break
         
#Cerramos la conexión:
sim.simxFinish(-1)